In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pointwiseclassification/classification_dataset.tsv
/kaggle/input/pointwiseclassificationcheckpoints/config.json
/kaggle/input/pointwiseclassificationcheckpoints/training_args.bin
/kaggle/input/pointwiseclassificationcheckpoints/tokenizer.json
/kaggle/input/pointwiseclassificationcheckpoints/tokenizer_config.json
/kaggle/input/pointwiseclassificationcheckpoints/pytorch_model.bin
/kaggle/input/pointwiseclassificationcheckpoints/special_tokens_map.json
/kaggle/input/pointwiseclassificationcheckpoints/vocab.txt
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-234522/config.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-234522/trainer_state.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-234522/training_args.bin
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-234522/tokenizer.json
/kaggle/input/pointwiseclassificationcheckpoints/checkpoint-234522/tokenizer_config.json
/kaggle/input/pointwiseclassificationcheckpoints/che

In [2]:
checkpoint = 'sentence-transformers/distiluse-base-multilingual-cased-v2'

In [3]:
!pip install datasets==2.1.0

     |████████████████████████████████| 325 kB 3.1 MB/s            
  Attempting uninstall: datasets
    Found existing installation: datasets 1.18.4
    Uninstalling datasets-1.18.4:
      Successfully uninstalled datasets-1.18.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.1 requires datasets<2.0,>=1.2.1, but you have datasets 2.1.0 which is incompatible.


In [4]:
classify_df = pd.read_csv('/kaggle/input/pointwiseclassification/classification_dataset.tsv', sep='\t')
classify_df.rename(columns={'esci_label': 'label'}, inplace=True)

In [5]:
from sklearn.model_selection import train_test_split
train = []
test = []
for label, _df in classify_df.groupby(['label']):
    train_df, test_df = train_test_split(_df,test_size=0.2, random_state=5)
    train.append(train_df)
    test.append(test_df)

In [6]:
train_df = pd.concat(train, ignore_index=True)
test_df = pd.concat(test, ignore_index=True)

In [7]:
scores = {'exact': 1.0, 'substitute': 0.1, 'complement': 0.01, 'irrelevant': 0.0}

In [8]:
upsampled_train = []
sample_frac = {'exact': 0.8, 'substitute': 1.0, 'irrelevant': 2.0, 'complement': 7.8}
for label, _df in train_df.groupby(['label']):
    upsampled_train.append(_df.sample(frac=sample_frac[label], replace=True, random_state=5))

In [9]:
upsample_train_df = pd.concat(upsampled_train, ignore_index=True)

In [10]:
upsample_train_df.label.value_counts()

complement    253032
exact         218347
substitute    214369
irrelevant    211290
Name: label, dtype: int64

In [11]:
pd.set_option('display.max_colwidth', None)

In [12]:
train_df[train_df['label'] == 'substitute'].sample(n=5)

,input,label
518172,そば インスタント jp [SEP] マルちゃん マルちゃん正麺 中華そば 和風醤油 ５食パック（105gX5P） ×6個 jp マルちゃん正麺 原材料:めん（小麦粉（国内製造）、植物油脂、食塩、植物性たん白、卵白）、添付調味料（しょうゆ、ラード、魚介エキス、食塩、チキンエキス、ポークエキス、香味油脂、砂糖、香辛料、酵母エキス）／加工でん粉、調味料（アミノ酸等）、トレハロース、かんすい、酒精、炭酸カルシウム、レシチン、酸化防止剤（ビタミンＣ、ビタミンＥ）、カラメル色素、クチナ\n内容量:5P×6個\n商品サイズ(高さx奥行x幅):15.5cm×38cm×35cm,substitute
612817,低周波治療器 オムロン jp [SEP] 低周波マッサージ器 マッサージ機 電気で筋肉に刺激 に痛み緩和治療デバイス 16種類モード 20段階調節 リラックス 血行促進 腰痛 疲れ コリ解決 神経痛 痛みの緩解 jp Duomai シルバー,substitute
450112,"escape room in a box us [SEP] Exit: The House of Riddles | Exit: The Game - A Kosmos Game from Thames & Kosmos | Family-Friendly, Card-Based At-Home Escape Room Experience for 1 To 4 Players, Ages 10+, Multi-colored us Thames & Kosmos Unique storyline: in this adventure from the exit: the game series, players are summoned to an abandoned house to solve a mystery, but upon arrival, they get locked Inside! Can you crack The case and escape?\nTeamwork: up to 4 players work together to solve a series of riddles and puzzles, crack codes, and escape!\nUnplugged: no app required! Everything needed to solve the riddles (including hints, if necessary) is included in the box.\nPlay time & difficulty level: This exit game, for ages 10+, takes 1-2 hours to complete, and has a difficulty rating of 2 out of 5.\nOne-time use: this game can be played only one time because you must Mark up, fold, and tear The game materials to solve the riddles and escape. Multi-colored",substitute
610320,"gaggia babila espresso machine us [SEP] Gaggia Cadorna Barista Plus Super-Automatic Espresso Machine, Black, Medium us Gaggia 6 Pre-Programmed Beverages|Build your home barista menu with 6 pre-programmed beverages including a ristretto, espresso, espresso lungo, a standard coffee, an Americano and hot water for tea, hot cocoa, or infusions.\n4 Personalized Profiles|Create up to 4 user profiles on the Cadorna Plus and save your favorite drinks for everyday consistency.\nQuick Heat Boiler|The boiler on the Cadorna Plus is made of lightweight aluminum and durable stainless steel allowing the machine to heat-up quickly without a long wait time.\nCommercial-Style Steam Wand|The Cadorna Plus features a commercial-style steam wand, perfect for making top quality milk foam for lattes, cappuccinos, flat whites and more.\n1 YEAR WARRANTY: Hassle free 1 year warranty handled by Gaggia directly. Black",substitute
518469,"ratones sin cable gaming es [SEP] NGS FLAME BLACK - Ratón Óptico 1000dpi con Cable USB, Ratón para Ordenador o Portátil con 3 Botones, Ambidiestro, Negro es NGS Ratón óptico de 1000 dpi de resolución con conexión USB para aquellos que exigen precisión en el movimiento.\nConexión a través de su cable USB y compatible con todas las plataformas.\nDispone de scroll vertical, 3 pulsadores y Thumb Control.\nErgonomía ambidiestra: puede ser utilizado tanto por usuarios diestros como zurdos.\nColor rojo. Negro",substitute


In [13]:
test_df.head(2)

,input,label
0,"tracxas 6wd us [SEP] for Traxxas TRX-4 Trail Defender Crawler / TRX-6 MERC-Benz G63 Upgrade Parts Steel 10T Pinion Gear - 1Pc Set Black us GPM For Traxxas TRX-4 Trail Defender Crawler / TRX-6 Merc-Benz G63 Upgrade Parts Steel 10T Pinion Gear - 1Pc Set Black Product Name : Steel 10T Pinion Gear - 1Pc Set\nMade from harden steel #45, strong and durable to achieve smooth performance. Suitable for all motors with below specifiaction - Middle hole : 3.175mm ; Tooth distance : 0.8mm ; Pitch : 32 ; Tooth : 10T.\nAccessories included : 3*3 Screw Shaft - 1pc.\nOriginal stock part number : 6747.\nWill fit : 1/10 Traxxas TRX-4 Defender Trail Crawler (82056-4), TRX-6 Mercedes-Benz G63 AMG 6X6 Crawler (88096-4)",complement
1,"caña baitcasting es [SEP] CHSEEO 48PCS Cebo de Pesca Señuelos de Pesca con Ganchos Cebos Artificial Swimbait Manivela Cebo Cucharillas Pesca Accesorios Aparejos De Pesca para la Pesca #2 es CHSEEO <br><b>CHSEEO Cebo de Pesca Señuelos de Pesca con Ganchos Cebos Artificial</b></br> <br>Todos los cebos son cuidadosamente seleccionados y satisfacen las necesidades de diferentes especies de peces que te atrapan. Y el diseño perfecto puede aumentar tus capturas y crear más diversión, que funciona bien en el uso práctico. Disfrutando tu tiempo de pesca. </br> <br><b>Caracteristicas:</b></br> <br>Señuelos son adecuados para el río y la pesca en el mar a diferentes profundidades. Adecuado para diferentes tipos de peces, como el bajo, la perca, la lucioperca, lucio, muskie, gobio, trucha, etc. Adecuado para la pesca de agua dulce y salada.</br> <br>Los ojos holográficos en 3D y las plumas avanzadas en ambos lados del cuerpo lo hacen fluir y nadar naturalmente en el agua, haciéndolo más realista. Los colores brillantes, vida-como las acciones de la natación en el agua para provocar depredador para morder. El cebo se encuentra bien en la superficie del agua y atrae a los peces.</br> <br>Hecho de Material de la más alta calidad. son muy robustos y sólidos,también no pesa mucho. Los ganchos están hechos de acero con alto contenido de carbono y son resistentes. </br> <br>Más cómodo y más seguro para la pesca al aire libre. Disfruta de la diversión de pesca con el set de anzuelos de pesca. Herramientas maravillosas para los amantes de la pesca. Ideal para Navidad, Acción de Gracias, Día del padre, etc.</br> <br><b>Lista de paquetes:</b></br> <br> 48 x Cebo de Pesca</br> <br> Atención: debido a diversos lotes, el color será un poco diferente. Por favor, tenga cuidado al usarlo, porque sus ganchos muy afilados.</br> Ampliamente Utilizado: 48 piezas de señuelos de pesca y accesorios. Señuelos son adecuados para el río y la pesca en el mar a diferentes profundidades. Adecuado para diferentes tipos de peces, como el bajo, la perca, la lucioperca, lucio, muskie, gobio, trucha, etc. Adecuado para la pesca de agua dulce y salada.\nDiseño Especial: Los ojos holográficos en 3D y las plumas avanzadas en ambos lados del cuerpo lo hacen fluir y nadar naturalmente en el agua, haciéndolo más realista. Los colores brillantes, vida-como las acciones de la natación en el agua para provocar depredador para morder. El cebo se encuentra bien en la superficie del agua y atrae a los peces.\nMaterial de Alta Calidad: Hecho de Material de la más alta calidad. son muy robustos y sólidos,también no pesa mucho. Los ganchos están hechos de acero con alto contenido de carbono y son resistentes.\nAnzuelos Triples Afilados: 2 anzuelos triples fuertes y afilados. Cada uno viene con 2 anzuelos triples afilados que tiene una potente herramienta de captura. El diseño perfecto puede aumentar sus capturas y crear más divertido.\nBonito Regalo: Más cómodo y más seguro para la pesca al aire libre. Disfruta de la diversión de pesca con el set de anzuelos de pesca. Herramientas maravillosas para los amantes de la pesca. Ideal para Navidad, Acción de Gracias, Día del padre, etc. 48PCS",complement


In [14]:
train_df.head(2)

,input,label
0,"tack cleaner for horses us [SEP] Pack of 12 Tack Sponges us Hydra Sponge Pack of 12 Tack Sponges VALUE PACK of 12 Creative Hobbies Best Quality Synthetic sponges for Painting, Face Painting, Crafts, Pottery, Clay, Household Use & more!\nNEW IMPROVED FORMULA Synthetic material is soft and absorbent, finishes clay smoothly, is gentle on faces for face painting, and very absorbent.\nShipped compressed to save space, sponges expand by rinsing in warm water or exposure to air after unpacking.\nVERSATILE - Great for face painting, leather tack cleaning and conditioning, applying paint in household or craft projects to washing dishes.",complement
1,"brother p-touch pt-d210 us [SEP] Khanka Hard Travel Case Replacement for Brother P-Touch PT-D210 Label Maker us khanka DISCLAIMER: Product only include Khanka brand casing. Does NOT include other products as pictured. We are NOT sponsored by other brands. Khanka Carrying Storage Organizer Case Bag Replacement for Brother P-Touch PT-D210 Label Maker fits 1/2-Inch Standard Laminated Tape\nExtra Separate Compartment Mesh Pocket for 1/2-Inch Standard Laminated Tape\nHand Strap For keeping your devices safe and handy,Lightweight. Packed inclued: One Case\nMaterial: EVA Color: Black External Size:7*6.6*3.5 inch Weight: 220g Black",complement


In [15]:
train_df.shape[0], test_df.shape[0]

(625388, 156350)

In [16]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 3.1 MB/s             
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120748 sha256=b9b206174513d9704b8c464615aa32d1567e54c6a975431b7bcddf7f9e125326
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [17]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer(checkpoint)
train_examples = []
test_examples = []
train_batch_size = 16
for sentence, label in zip(train_df.input.to_list(), train_df.label.to_list()):
    sentence_list = sentence.split(' [SEP] ')
    train_examples.append(InputExample(texts=sentence_list, label=scores[label]))
for sentence, label in zip(test_df.input.to_list(), test_df.label.to_list()):
    sentence_list = sentence.split(' [SEP] ')
    #print(sentence_list)
    test_examples.append(InputExample(texts=sentence_list, label=scores[label]))

train_dataset = SentencesDataset(train_examples, model)
test_dataset = SentencesDataset(test_examples, model)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=2 * train_batch_size)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [18]:
import math
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
# Configure the training
model_save_path = './'
num_epochs = 3
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
print("Warmup-steps: {}".format(warmup_steps))
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_examples, name='test_pointwise')

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=model_save_path
         )

Warmup-steps: 11727


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/39087 [00:00<?, ?it/s]

Iteration:   0%|          | 0/39087 [00:00<?, ?it/s]

Iteration:   0%|          | 0/39087 [00:00<?, ?it/s]